In [1]:
import argparse
import h5py

import numpy as np
from neuropacks import NHP
from sem import SEMSolver
from pyuoi.linear_model import UoI_Lasso

In [2]:
data_path = '/Users/psachdeva/data/NHP/indy_20160411_01.mat'
bin_width = 0.5
region = 'M1'
transform = 'square_root'
method = 'UoI_Lasso'
estimation_score = 'r2'
results_path = 'test.h5'
results_group = 'Lasso'

In [3]:
# create data extraction object
nhp = NHP(data_path=data_path)

# get response matrix
Y = nhp.get_response_matrix(
    bin_width=0.5,
    region='M1',
    transform='square_root'
)

# create solver object
solver = SEMSolver(Y=Y)

In [4]:

# perform cross-validated coupling fits
results = solver.estimation(
    model='c',
    method=method,
    targets=0,
    n_folds=10,
    random_state=2332,
    metrics=['r2', 'BIC', 'AIC'],
    verbose=True,

    # general options
    fit_intercept=True,
    normalize=True,

    # Lasso specific
    cv=10,
    max_iter=5000,

    # UoI Lasso specific
    n_boots_sel=30,
    n_boots_est=30,
    selection_frac=0.8,
    estimation_frac=0.8,
    n_lambdas=48,
    stability_selection=1,
    estimation_score=estimation_score
)

results_file = h5py.File(results_path, 'a')
group = results_file.create_group(results_group)
# place results in group
for key in results.keys():
    # need to handle training and test folds separately
    if key == 'training_folds' or key == 'test_folds':
        folds_group = group.create_group(key)
        for fold_key, fold_val in results[key].items():
            folds_group[fold_key] = fold_val
    else:
        group[key] = results[key]

results_file.close()

Fold 0


AttributeError: 'tuple' object has no attribute 'keys'

In [5]:
f = h5py.File('fits/nhp/indy_20160411_01.h5', 'r')

In [6]:
training_idx = f['Lasso/training_folds/fold_0'][:]
test_idx = f['Lasso/test_folds/fold_0'][:]

In [7]:
f.close()

In [8]:
nhp = NHP(data_path='/Users/psachdeva/data/NHP/indy_20160411_01.mat')

In [9]:
Y_new = nhp.get_response_matrix(bin_width=0.5, region='M1', transform='square_root')

In [10]:
X_alt = np.delete(Y, 0, axis=1)[training_idx, :]
y_alt = Y[:, 0][training_idx]

In [11]:
X_train, y_train = solver.fit_coupling(
    targets=np.array([0]),
    method='UoI_Lasso',
    training_idx=training_idx,
    test_idx=test_idx,
    # general options
    fit_intercept=True,
    normalize=True,

    # Lasso specific
    cv=10,
    max_iter=1000,

    # UoI Lasso specific
    n_boots_sel=30,
    n_boots_est=30,
    selection_frac=0.8,
    estimation_frac=0.8,
    n_lambdas=48,
    stability_selection=1,
    estimation_score=estimation_score
)

In [12]:
uoi = UoI_Lasso(
    normalize=True,
    n_boots_sel=5,
    n_boots_est=1,
    selection_frac=0.8,
    estimation_frac=0.8,
    n_lambdas=5,
    stability_selection=1,
    estimation_score=estimation_score,
    max_iter=1000,
    random_state=1234,
    warm_start=True
)
uoi.fit(X_alt, y_alt)

AttributeError: can't set attribute

In [14]:
uoi.selection_lm.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=1234,
   selection='cyclic', tol=0.0001, warm_start=True)

In [17]:
from sklearn.base import clone

In [19]:
clone(uoi.selection_lm)

AttributeError: 'Lasso' object has no attribute 'coef_'

In [12]:
uoi = UoI_Lasso(
    normalize=True,
    n_boots_sel=5,
    n_boots_est=1,
    selection_frac=0.8,
    estimation_frac=0.8,
    n_lambdas=5,
    stability_selection=1,
    estimation_score=estimation_score,
    max_iter=1000,
    random_state=1234,
    warm_start=True
)
uoi.fit(X_train, y_train)


---- 0
-0.014135138491563566
2
{'alpha': 0.007201325550952879}
[-0.5416395   0.1904113  -0.12742594 -0.12742594 -0.12742594  0.4583605
 -1.5416395  -0.12742594  0.69442847  0.1904113  -0.5416395   0.4583605
 -0.12742594  0.1904113   0.1904113   0.69442847  1.28678762 -1.5416395
 -0.5416395  -1.5416395  -1.5416395   0.69442847 -0.12742594  0.1904113
 -0.5416395   1.28678762  1.62063816  0.1904113   0.69442847 -0.12742594
 -1.5416395  -0.5416395   0.4583605   0.69442847  1.10411181  1.28678762
  0.4583605   0.4583605   1.28678762  0.69442847  1.28678762 -1.5416395
  1.62063816 -1.5416395  -0.12742594 -0.5416395  -1.5416395  -1.5416395
 -0.5416395   1.10411181  1.10411181 -0.12742594  0.69442847  0.4583605
  0.69442847 -1.5416395   0.90785024  0.90785024  1.62063816  0.1904113
 -0.5416395   0.90785024  0.1904113   0.4583605  -0.5416395  -0.5416395
  1.4583605   0.4583605   0.69442847 -1.5416395   1.10411181 -0.12742594
  1.62063816 -0.5416395   0.4583605  -1.5416395  -0.5416395   1.620638

 -0.12742594 -0.5416395 ]
===
0.016881135815936518
74
{'alpha': 7.20132555095288e-06}
[-1.5416395   2.20001788  0.1904113   0.90785024 -0.12742594  0.1904113
  0.1904113   1.10411181 -1.5416395  -0.12742594  1.62063816 -1.5416395
  0.1904113   1.28678762  0.69442847  0.1904113   0.4583605  -1.5416395
  1.62063816 -0.12742594  0.69442847  0.90785024 -1.5416395  -0.12742594
  0.4583605   0.90785024  0.1904113   0.4583605  -0.5416395  -0.5416395
  0.1904113   0.1904113  -1.5416395  -0.12742594  0.1904113   0.4583605
 -1.5416395   1.10411181 -0.5416395  -1.5416395  -1.5416395   0.4583605
 -0.5416395   0.90785024 -1.5416395  -0.5416395  -0.5416395  -0.12742594
  1.28678762 -1.5416395  -1.5416395  -0.5416395  -0.5416395  -0.5416395
  0.4583605   0.4583605   1.4583605  -0.12742594 -0.5416395   0.1904113
 -0.5416395   1.10411181 -0.5416395  -0.5416395   0.69442847  0.69442847
  0.69442847  0.69442847  2.06391177 -1.5416395  -0.5416395  -1.5416395
  0.4583605  -0.12742594 -0.12742594 -1.5416395

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


-0.02285521285096473
1000
{'alpha': 0.007201325550952879}
[-0.5416395   0.4583605   0.4583605   0.1904113  -0.12742594  0.1904113
  2.06391177  1.62063816 -0.12742594 -0.5416395  -0.12742594  0.4583605
  0.90785024  1.62063816  0.1904113  -1.5416395   1.4583605   1.10411181
  0.69442847  0.69442847 -0.12742594 -0.5416395   0.69442847 -0.12742594
 -1.5416395   0.1904113  -0.12742594 -1.5416395   1.28678762  0.1904113
  0.1904113   0.1904113   0.4583605   0.90785024  2.58146612 -0.5416395
  0.1904113   0.69442847  1.28678762  1.4583605   0.1904113  -0.12742594
 -0.12742594  2.33134384  0.4583605   0.4583605  -0.12742594  0.1904113
  0.90785024  0.4583605  -0.5416395   0.90785024  1.28678762  0.1904113
 -1.5416395  -0.12742594  0.1904113  -1.5416395  -0.5416395  -1.5416395
  0.4583605   0.90785024 -0.12742594  0.90785024 -0.5416395  -0.5416395
 -0.5416395   1.10411181  1.62063816  0.1904113   0.4583605  -1.5416395
 -0.12742594  0.90785024  1.77498529 -1.5416395   0.69442847  0.90785024
  

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


-0.012980721605169537
1000
{'alpha': 0.0012805968952246708}
[-0.5416395   0.4583605   0.4583605   0.1904113  -0.12742594  0.1904113
  2.06391177  1.62063816 -0.12742594 -0.5416395  -0.12742594  0.4583605
  0.90785024  1.62063816  0.1904113  -1.5416395   1.4583605   1.10411181
  0.69442847  0.69442847 -0.12742594 -0.5416395   0.69442847 -0.12742594
 -1.5416395   0.1904113  -0.12742594 -1.5416395   1.28678762  0.1904113
  0.1904113   0.1904113   0.4583605   0.90785024  2.58146612 -0.5416395
  0.1904113   0.69442847  1.28678762  1.4583605   0.1904113  -0.12742594
 -0.12742594  2.33134384  0.4583605   0.4583605  -0.12742594  0.1904113
  0.90785024  0.4583605  -0.5416395   0.90785024  1.28678762  0.1904113
 -1.5416395  -0.12742594  0.1904113  -1.5416395  -0.5416395  -1.5416395
  0.4583605   0.90785024 -0.12742594  0.90785024 -0.5416395  -0.5416395
 -0.5416395   1.10411181  1.62063816  0.1904113   0.4583605  -1.5416395
 -0.12742594  0.90785024  1.77498529 -1.5416395   0.69442847  0.90785024


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


-0.007566912825437835
1000
{'alpha': 0.00022772590913378038}
[-0.5416395   0.4583605   0.4583605   0.1904113  -0.12742594  0.1904113
  2.06391177  1.62063816 -0.12742594 -0.5416395  -0.12742594  0.4583605
  0.90785024  1.62063816  0.1904113  -1.5416395   1.4583605   1.10411181
  0.69442847  0.69442847 -0.12742594 -0.5416395   0.69442847 -0.12742594
 -1.5416395   0.1904113  -0.12742594 -1.5416395   1.28678762  0.1904113
  0.1904113   0.1904113   0.4583605   0.90785024  2.58146612 -0.5416395
  0.1904113   0.69442847  1.28678762  1.4583605   0.1904113  -0.12742594
 -0.12742594  2.33134384  0.4583605   0.4583605  -0.12742594  0.1904113
  0.90785024  0.4583605  -0.5416395   0.90785024  1.28678762  0.1904113
 -1.5416395  -0.12742594  0.1904113  -1.5416395  -0.5416395  -1.5416395
  0.4583605   0.90785024 -0.12742594  0.90785024 -0.5416395  -0.5416395
 -0.5416395   1.10411181  1.62063816  0.1904113   0.4583605  -1.5416395
 -0.12742594  0.90785024  1.77498529 -1.5416395   0.69442847  0.90785024

 -0.5416395   0.1904113 ]
===
-0.00815315804033975
64
{'alpha': 4.049602953449963e-05}
[ 0.4583605  -1.5416395  -1.5416395  -1.5416395   0.69442847 -1.5416395
 -0.5416395   0.69442847  0.69442847  0.69442847  1.28678762 -0.5416395
  1.28678762  1.4583605  -0.12742594  0.4583605   0.4583605  -0.5416395
 -1.5416395  -0.5416395   0.1904113  -1.5416395   0.90785024 -0.5416395
 -0.5416395  -1.5416395  -0.12742594  1.4583605   0.4583605  -0.5416395
  0.1904113   0.1904113   0.4583605   1.10411181 -0.5416395  -0.5416395
 -0.12742594  0.4583605   0.69442847 -0.12742594 -0.12742594  1.77498529
  0.69442847  0.1904113  -0.12742594 -0.5416395  -0.5416395  -0.12742594
 -0.12742594  1.10411181  1.62063816 -1.5416395   0.69442847  0.4583605
 -0.12742594 -1.5416395   0.1904113   0.69442847  2.06391177 -0.12742594
  1.77498529  1.10411181  1.4583605  -0.5416395   0.1904113  -0.12742594
  1.28678762  0.69442847  0.1904113  -0.12742594 -0.12742594  0.90785024
 -0.12742594  0.4583605   1.62063816 -0.1274

UoI_Lasso(comm=None, copy_X=True, eps=0.001, estimation_frac=0.8,
     estimation_score='r2', fit_intercept=True, max_iter=None,
     n_boots_est=1, n_boots_sel=5, n_lambdas=5, normalize=True,
     random_state=<mtrand.RandomState object at 0x1a162d4360>,
     selection_frac=0.8, stability_selection=1, warm_start=None)

In [18]:
191*0.2

38.2